<a href="https://colab.research.google.com/github/MLenthusiastic/Solution/blob/master/Context_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from torch.utils.data import Dataset
import json
import numpy as np
import torch.nn as nn
import cv2
import matplotlib.pyplot as plt
import seaborn as sns;sns.set()
from google.colab.patches import cv2_imshow
from IPython.display import Image, display
import torch
import math
import torch.nn.functional as F
from argparse import ArgumentParser
import copy
import torch.nn.functional as F
from PIL import Image

from torch.utils.tensorboard import SummaryWriter
import tensorflow as tf
import tensorboard as tb
tf.io.gfile = tb.compat.tensorflow_stub.io.gfile


parser = ArgumentParser(description='Solution Network')
parser.add_argument('--batch_size', type=int, default=16)
parser.add_argument('--constractive_loss_margin', type=float, default=0.8)
parser.add_argument('--learning_rate', type=float, default=1e-4)
parser.add_argument('--num_epochs', type=int, default=20)
parser.add_argument('--weight_decay', type=float, default=1e-5)
parser.add_argument('--mode', type=str, default='train')
parser.add_argument('--device', type=str, default='cuda')
parser.add_argument('--img_size', type=int, default=200)
parser.add_argument('--no_of_samples_per_class', type=int, default=100)
parser.add_argument('--no_of_classes', type=int, default=50)
parser.add_argument('--projector_img_size', type=int, default=32)

args, unknown = parser.parse_known_args()

DEVICE = args.device
if not torch.cuda.is_available():
    DEVICE = 'cpu'

#previous trained encoder model
class Encoder(nn.Module):

    def __init__(self):
        super(Encoder, self).__init__()

        #conv and fc works as encoder
        self.conv = nn.Sequential(nn.Conv2d(in_channels=3, out_channels=32, kernel_size=5),
                                  nn.ReLU(),
                                  nn.MaxPool2d(kernel_size=2, stride=2),
                                  nn.Conv2d(in_channels=32, out_channels=64, kernel_size=5),
                                  nn.ReLU(),
                                  nn.MaxPool2d(kernel_size=2, stride=2),
                                  nn.Conv2d(in_channels=64, out_channels=128, kernel_size=5),
                                  nn.ReLU(),
                                  nn.MaxPool2d(kernel_size=2, stride=2)
                                  )
        
        # output 128, 21, 21 
        self.fc = nn.Sequential(nn.Linear(128 * 21 * 21, 1024),
                                nn.ReLU(),
                                nn.Linear(1024, 1024),
                                nn.ReLU(),
                                nn.Linear(1024, 128)
                                )
        
    def forward(self, x):
        x = self.conv(x)
        x = x.view(x.size()[0], -1)
        z = self.fc(x)
        return z

class ContextDavis(Dataset):

  def __init__(self, json_data, memmap):
      self.memmap = memmap
      self.json_data = json_data
      self.shape = self.json_data["shape"]
      self.objects = self.json_data["objects"]
      self.contexts = self.json_data["contexts"]
      self.keys_of_images = list(self.contexts.keys())
      self.keys_of_images.sort()
      self.start_img_index = int(self.keys_of_images[0])
      self.length = len(self.keys_of_images)

  def __getitem__(self, index):
    correct_index = self.start_img_index + index
    contexts_per_image = self.contexts[str(correct_index)]
    
    img_objects = []
    img_classes = []
    for context_image in contexts_per_image:
      img_object = self.objects[str(context_image)]
      img_object_width = img_object['width']
      img_object_height = img_object['height']
      image_ =  self.memmap[context_image, :, :img_object_width, :img_object_height].astype(np.float32)
      class_ = img_object['class']

      image_tensor_ext = torch.from_numpy(image_).unsqueeze(dim=0)
      image_tensor = F.interpolate(image_tensor_ext, size=(args.img_size,args.img_size))
      image_tensor = image_tensor.squeeze(dim=1)

      img_objects.append(image_tensor)
      img_classes.append(class_)

    return [img_objects, img_classes]

  def __len__(self):
      return self.length


folder_path = './drive/My Drive/'
with open(folder_path+'test_davis_json.txt') as json_file:
    davis_json = json.load(json_file)

shape = davis_json["shape"]
memmap_path = folder_path+'test_davis.mmap'
davis_memmap = np.memmap(memmap_path, dtype='uint8', mode='r', shape=tuple(shape))

davis_dataset = ContextDavis(davis_json,davis_memmap)
davis_dataloader = torch.utils.data.DataLoader(davis_dataset, batch_size=1, shuffle=False)
#shuflling changes order and keep batch size 1

#load previously trained encoder model
model_path = './drive/My Drive/encoder_2.pth'
encoder_model = Encoder()
encoder_model.load_state_dict(torch.load(model_path))
encoder_model.eval()

for epoch in range(1,args.num_epochs+1):

  for batch in davis_dataloader:

      img_objects, classes = batch[0] , batch[1]
      z_vectors = []
      for img_object in img_objects:
          z_vectors.append(encoder_model(img_object.squeeze(dim=1)))

      


      
      #break



'''
data = davis_dataset.__getitem__(0)
imgs, clss  = data
for i in imgs:
  j = i[0].numpy().transpose(2,1,0)
  cv2_imshow(j)
'''
'''
batch = next(iter(davis_dataloader))

print(len(batch))
print(batch[0][0]) # for images
print(batch[1][0]) #for classes

imges = batch[0][0]
for i in imges:
  j = i[0].numpy().transpose(2,1,0)
  cv2_imshow(j)
'''